In [91]:
import TAD_SIM as ts
import numpy as np
reload(ts)


No. of tads:
[100]
No. of chrms labeled/hybe:
[14 15 16 17 18] 15.9722222222 0.622990597


In [2]:
import time,os,glob
import TAD_SIM as ts
import numpy as np
import cPickle as pickle
reload(ts)

dic_parms = {'num_chr':23,'num_hom':2,'num_TADs':300,'chr_cutoff':0.,'num_label':16,'miss_rate':0.0325,'cell_id':0}
#simulate a chr ter
for chr_cutoff in [100.]:
    dic_parms['chr_cutoff']=chr_cutoff
    start = time.time()
    chrters=ts.chromosomes(nuc_dia=10000,pixel_sz=100,nchr=dic_parms['num_chr']*dic_parms['num_hom'],plt_val=False,
                           return_im=False,cutoff=dic_parms['chr_cutoff'])
    print "Simulated chr teritorries. Time(s): "+str(time.time()-start)

    #simulate single cells
    for cell_id in range(10):
        start = time.time()
        cell=[]
        for chrter in chrters:
            tads_=ts.TAD_generator(chrter,noTADs=dic_parms['num_TADs'],udist=-0.44276236166846844,sigmadist=0.57416477624326434,
                                   nuc_dia=10000,pix_sz=100)
            cell.append(tads_)
        cell=np.array(cell)
        dic_parms['cell_id'] = cell_id
        print "Simulated cell. Time(s): "+str(time.time()-start)

        for num_label in [12,14,16,20,22]:
            #decide on encoder
            start = time.time()
            dic_parms['num_label'] = num_label
            hybes=ts.code_encoder(nchr=dic_parms['num_chr'],ntads=dic_parms['num_TADs'],nlabel_=dic_parms['num_label'],
                                  no_hom=dic_parms['num_hom'])
            print "Created encoder. Time(s): "+str(time.time()-start)

            #simulate data from a cell
            start = time.time()
            im_data,truth=ts.simulated_imdata(hybes, cell, err_rate=dic_parms['miss_rate'])
            print "Simulated data. Time(s): "+str(time.time()-start)

            #decode up to homology
            start = time.time()
            corr,incorr,interp=ts.decoder(im_data,hybes,truth,n_chr=dic_parms['num_chr'])
            corr_first,incorr_first,interp_first = map(np.array,interp),corr,incorr

            print "First decoding. %Incorrect: "+str(incorr/float(corr+incorr)*100)
            for iter_ in range(3):
                corr,incorr,interp=ts.refine_decoder(im_data,hybes,interp,truth,n_chr=dic_parms['num_chr'],
                                                     noTads=dic_parms['num_TADs'],fr_nn=0.8)
                print "Refining decoding - step "+str(iter_)+". %Incorrect: "+str(incorr/float(corr+incorr)*100)
            corr_last,incorr_last,interp_last = map(np.array,interp),corr,incorr
            print "Decoded up to homology. Time(s): "+str(time.time()-start)

            #decode hom chrs
            start = time.time()
            corrsep,incorrsep,interpsep = ts.separator(im_data,interp,truth,num_chr=dic_parms['num_chr'],num_hom=dic_parms['num_hom'])

            print "Separated homologs. Time(s): "+str(time.time()-start)
            dic_keep={}
            dic_keep['cell']=cell
            dic_keep['encoder']=hybes
            dic_keep['sim_data']=im_data
            dic_keep['truth']=truth

            dic_keep['interp_first']=interp_first
            dic_keep['corr_first']=corr_first
            dic_keep['incorr_first']=incorr_first

            dic_keep['interp_last']=interp_last
            dic_keep['corr_last']=corr_last
            dic_keep['incorr_last']=incorr_last

            dic_keep['interp_sep']=interpsep
            dic_keep['corr_sep']=corrsep
            dic_keep['incorr_sep']=incorrsep
            dic_keep['dic_parms']=dic_parms

            filename = '__'.join([key+'_'+str(dic_parms[key]) for key in np.sort(dic_parms.keys())])+'.pkl'
            save_folder = 'save_folder'
            if not os.path.exists(save_folder):
                os.mkdir(save_folder)
            pickle.dump(dic_keep,open(save_folder+os.sep+filename,'wb'))

Simulated chr teritorries. Time(s): 19.4049999714
Simulated cell. Time(s): 4.07299995422
No. of tads:
[100]
No. of chrms labeled/hybe:
[10 11 12 13 14 15] 11.9791666667 0.676682573705
Created encoder. Time(s): 0.328000068665
Simulated data. Time(s): 0.0160000324249
First decoding. %Incorrect: 3.21276117726
Refining decoding - step 0. %Incorrect: 2.13435183105
Refining decoding - step 1. %Incorrect: 2.02201752415
Refining decoding - step 2. %Incorrect: 2.02201752415
Decoded up to homology. Time(s): 65.2879998684
Separated homologs. Time(s): 15.1600000858
No. of tads:
[100]
No. of chrms labeled/hybe:
[11 12 13 14 15] 13.9393939394 0.666942091866
Created encoder. Time(s): 0.225000143051
Simulated data. Time(s): 0.0149998664856
First decoding. %Incorrect: 5.34747292419
Refining decoding - step 0. %Incorrect: 3.22653429603
Refining decoding - step 1. %Incorrect: 3.09115523466
Refining decoding - step 2. %Incorrect: 3.31678700361
Decoded up to homology. Time(s): 71.1340000629
Separated homol

In [5]:
dic_parms = {'num_chr':23,'num_hom':2,'num_TADs':100,'chr_cutoff':0.,'num_label':16,'miss_rate':0.0325}

In [16]:
6*10*6

360

In [17]:
import os

In [3]:
import time
import TAD_SIM as ts
import numpy as np
reload(ts)


#simulate a cell
start = time.time()
chrters=ts.chromosomes(nuc_dia=10000,pixel_sz=100,nchr=46,plt_val=False,return_im=False,cutoff=0.)


cell=[]
for chrter in chrters:
    tads_=ts.TAD_generator(chrter,noTADs=100,udist=-0.44276236166846844,sigmadist=0.57416477624326434,nuc_dia=10000,pix_sz=100)
    cell.append(tads_)
cell=np.array(cell)
print "Simulated cell. Time(s): "+str(time.time()-start)

#decide on encoder
start = time.time()
hybes=ts.code_encoder(nchr=23,ntads=100,nlabel_=16,no_hom=2)
print "Created encoder. Time(s): "+str(time.time()-start)

#simulate data from a cell
start = time.time()
im_data,truth=ts.simulated_imdata(hybes, cell, err_rate=0.0325)
dic_keep['cell']=cell
dic_keep['encoder']=hybes
dic_keep['sim_data']=im_data
dic_keep['truth']=truth
print "Simulated data. Time(s): "+str(time.time()-start)

#decode up to homology
start = time.time()
corr,incorr,interp=ts.decoder(im_data,hybes,truth,n_chr=23)
dic_keep['interp_first']=interp
dic_keep['corr_first']=corr
dic_keep['incorr_first']=incorr

print "First decoding. %Incorrect: "+str(incorr/float(corr+incorr)*100)
for iter_ in range(3):
    corr,incorr,interp=ts.refine_decoder(im_data,hybes,interp,truth,n_chr=23,noTads=100,fr_nn=0.8)
    print "Refining decoding - step "+str(iter_)+". %Incorrect: "+str(incorr/float(corr+incorr)*100)
dic_keep['interp_last']=interp
dic_keep['corr_last']=corr
dic_keep['incorr_last']=incorr
print "Decoded up to homology. Time(s): "+str(time.time()-start)

#decode hom chrs
start = time.time()
corr,incorr,interpsep = ts.separator(im_data,interp,truth,num_chr=23,num_hom=2)
dic_keep['interpsep']=interpsep
dic_keep['corrsep']=corr
dic_keep['incorrsep']=incorr
print "Separated homologs. Time(s): "+str(time.time()-start)


Simulated chr teritorries. Time(s): 24.736000061
Simulated cell. Time(s): 4.37999987602
No. of tads:
[100]
No. of chrms labeled/hybe:
[15 16 17 18] 15.9722222222 0.600282855138
Created encoder. Time(s): 0.0780000686646
Simulated data. Time(s): 0.0
First decoding. %Incorrect: 3.96128741841
Refining decoding - step 0. %Incorrect: 1.57551204141
Refining decoding - step 1. %Incorrect: 1.53049741166
Refining decoding - step 2. %Incorrect: 1.53049741166
Decoded up to homology. Time(s): 85.0750000477


NameError: global name 'num_chr' is not defined

In [4]:
reload(ts)
start = time.time()
corr,incorr,interpsep = ts.separator(im_data,interp,truth,num_chr=23,num_hom=2)
dic_keep['interpsep']=interpsep
dic_keep['corrsep']=corr
dic_keep['incorrsep']=incorr
print "Separated homologs. Time(s): "+str(time.time()-start)

Separated homologs. Time(s): 18.3229999542


In [125]:
reload(ts)
interpsep = ts.separator(im_data,interp,num_chr=23,num_hom=2)

In [124]:
def compare_to_truth(interpsep,truth,num_hom=2):
    pairs = []
    for hb in range(len(interpsep)):
        pairs.extend(zip(interpsep[hb],truth[hb]))
    chrs_pairs = ts.partition_map(pairs,[pair[-1]%num_chr for pair in pairs])

    import itertools
    permuts = np.array(list(itertools.permutations(range(num_hom))))
    
    total = np.sum(map(len,truth))
    correct = 0
    interpsep_ = map(np.array,interpsep)
    for chr_pairs in chrs_pairs:
        chr_pairs = np.array(chr_pairs,dtype=int)

        id_chrs_truth = np.unique(chr_pairs[:,1])

        perm_values = []
        for permut in permuts:
            id_chrs_data = id_chrs_truth[permut]
            perm_values.append(np.sum([np.sum((chr_pairs[:,0]==idd)*(chr_pairs[:,1]==idt)) for idd,idt in zip(id_chrs_data,id_chrs_truth)]))
        best_permut = permuts[np.argmax(perm_values)]
        id_chrs_data = id_chrs_truth[permut]
        for idd,idt in zip(id_chrs_data,id_chrs_truth):
            interpsep_ = [interpsep__==3for interpsep__ in interpsep_]
            
        correct+=np.max(perm_values)
    return correct,total-correct
compare_to_truth(interpsep,truth,num_hom=2)

(4402, 44)

In [130]:
arr=np.array([1,2,3,4,4,4,2,2,2,3])
arr[arr==2]=-1
arr

array([ 1, -1,  3,  4,  4,  4, -1, -1, -1,  3])

In [120]:
id_chrs_truth

array([ 0, 23])

In [103]:
num_chr=23
#num_hom=2


In [45]:
point_col = ts.flatten(im_data)
chr_col = ts.flatten(interp)
point_part = ts.partition_map(point_col,chr_col)

chr0pts = np.array(point_part[0])
import matplotlib.pyplot as plt
plt.plot(chr0pts[:,0],chr0pts[:,1],'o')


point_col = ts.flatten(im_data)
chr_col = ts.flatten(interp2)
point_part = ts.partition_map(point_col,chr_col)

chr0pts = np.array(point_part[0])
import matplotlib.pyplot as plt
plt.plot(chr0pts[:,0],chr0pts[:,1],'o')
plt.show()

In [52]:
corr,corr2,corr3,corr4

(4259, 4388, 4389, 4392)

In [50]:
corr+incorr,corr2+incorr2

(4439, 4439)

In [36]:
possible_chrs_hybes=[]
for hybe in hybes:
    possible_chrs_hybes.append(np.where(hybe>0)[0])

In [83]:
truth,interp_hom

([[0,
   1,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36],
  [0,
   1,
   2,
   3,
   5,
   6,
   7,
   8,
   9,
   16,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   28,
   29,
   30,
   31,
   32,
   39,
   41,
   42,
   43,
   44,
   45],
  [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   15,
   16,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   38,
   39,
   41,
   42,
   43,
   44,
   45],
  [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   9,
   15,
   16,
   17,
   18,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   32,
   38,
   39,
   41,
   43,
   44,
   45],
  [0,
   1,
   3,
   4,
   5,
   6,
   8,
   9,
   11,
   15,
   16,
   17,
   19,
   20,
   22,
   23,
   24,
   26,
   27,
   28,
   29,
   31,
   32,
   33,
   34,
   38,
   3

In [77]:
reload(ts)
def separator(im_data,interp,num_chr=23,num_hom=2):
    im_data_enhanced = map(list,im_data)
    for ihybe in range(len(im_data_enhanced)):
        im_data_enhanced_hybe = im_data_enhanced[ihybe]
        for ipoint in range(len(im_data_enhanced_hybe)):
            point = im_data_enhanced_hybe[ipoint]
            point = list(point)+[ihybe,ipoint]
            im_data_enhanced_hybe[ipoint] = point
        im_data_enhanced[ihybe] = im_data_enhanced_hybe
    #iterate through chromosomes to patition the data
    pts_chrs,chr_hybes = [],[]
    for pts_hybe,chr_hybe in zip(im_data_enhanced,interp):
        pts_chr_ = ts.partition_map(pts_hybe,chr_hybe)
        chr_hybe_ = np.unique(chr_hybe)
        pts_chrs.extend(pts_chr_)
        chr_hybes.extend(chr_hybe_)
    pts_partitioned = ts.partition_map(pts_chrs,chr_hybes)
    chrs_ids = np.unique(chr_hybes)

    interp_hom = map(np.array,interp)
    #iterate through chromosome ids
    for chrs_id,pts_partitioned_ in zip(chrs_ids,pts_partitioned):
        #chrs_id = chrs_ids[1]
        #pts_partitioned_ = pts_partitioned[1]

        id_hybe_start = np.argsort(map(len,pts_partitioned_))[-1]
        pts_start = pts_partitioned_[id_hybe_start]
        assert len(pts_start)==num_hom
        #split_chr = split_dic[chrs_id]
        #for ipt,pt in enumerate(pts_start):
        #    split_chr[ipt].append(pt)

        pts_tobeasigned = list(pts_partitioned_)
        pts_tobeasigned.pop(id_hybe_start)

        chr_estim = [[list(val)+[ival]for ival,val in enumerate(pts_partitioned_[id_hybe_start])]]

        for pts_hybe in pts_tobeasigned:
            mean_dists_hybe = []
            chr_estim_flat = np.array(ts.flatten(chr_estim))
            split_chr = ts.partition_map(chr_estim_flat,chr_estim_flat[:,-1])
            for pt in pts_hybe:
                mean_dists = [np.median([np.sqrt(np.sum((pt[:3]-pt_t[:3])**2)) for pt_t in split_]) for split_ in split_chr]
                #mean_dists has num of homolog dists
                mean_dists_hybe.append(mean_dists)
            picks = ts.unique_classif(mean_dists_hybe)
            chr_estim.append([list(pts_hybe[pick[0]][:])+[pick[1]] for pick in picks])

        chr_estim_new = chr_estim
        while True:
            no_flips,chr_estim_new = refine_separator(chr_estim_new)
            if no_flips==0:
                break
        #Return the sames as interp(chromosme_id_all) but dealt with degeneracy
        for pt in ts.flatten(chr_estim_new):
            interp_hom[pt[3]][pt[4]]+=pt[-1]*num_chr
    return interp_hom
def refine_separator(chr_estim):
    chr_estim_flat = np.array(ts.flatten(chr_estim))
    split_chr = ts.partition_map(chr_estim_flat,chr_estim_flat[:,-1])
    chr_estim_new=[]
    for pts_hybe in chr_estim:
        mean_dists_hybe = []
        for pt in pts_hybe:
            mean_dists = [np.median([np.sqrt(np.sum((pt[:3]-pt_t[:3])**2)) for pt_t in split_]) for split_ in split_chr]
            #mean_dists has num of homolog dists
            mean_dists_hybe.append(mean_dists)
        picks = ts.unique_classif(mean_dists_hybe)
        chr_estim_new.append([list(pts_hybe[pick[0]][:])+[pick[1]] for pick in picks])

    chr_estim_flat = np.array(ts.flatten(chr_estim))
    chr_estim_flat_new = np.array(ts.flatten(chr_estim_new))  
    reind_old = np.argsort(chr_estim_flat[:,0])
    reind_new = np.argsort(chr_estim_flat_new[:,0])
    pos_dif = np.where(chr_estim_flat[reind_old,-1]!=chr_estim_flat_new[reind_new,-1])[0]
    no_flips = len(pos_dif)
    return no_flips,chr_estim_new

In [80]:
np.unique(ts.flatten(interp_hom))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], dtype=int64)

In [75]:
interp_hom = map(np.array,interp)
for pt in ts.flatten(chr_estim_new):
    interp_hom[pt[3]][pt[4]]+=pt[-1]*num_chr

In [76]:
interp_hom

[array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,  0, 24,
         2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13], dtype=int64),
 array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 16, 18, 19, 20, 21, 22,  0, 24,
         2,  3,  5,  6,  7,  8,  9, 16, 18, 19, 20, 21, 22], dtype=int64),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 15, 16, 18, 19, 20, 21, 22,  0,
        24,  2,  3,  4,  5,  6,  7,  8, 15, 16, 18, 19, 20, 21, 22], dtype=int64),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 15, 16, 17, 18, 20, 21, 22,  0,
        24,  2,  3,  4,  5,  6,  7,  9, 15, 16, 18, 20, 21, 22], dtype=int64),
 array([ 0,  1,  3,  4,  5,  6,  8,  9, 11, 15, 16, 17, 19, 20, 22,  0, 24,
         3,  4,  5,  6,  8,  9, 10, 11, 15, 16, 17, 19, 20, 22], dtype=int64),
 array([ 0,  1,  2,  3,  4,  5,  6,  8, 10, 11, 14, 16, 17, 19, 21, 22,  0,
        24,  2,  3,  4,  5,  6,  8, 10, 14, 16, 17, 19, 21, 22], dtype=int64),
 array([ 0,  1,  2,  3,  4,  5,  6,  9, 11, 12, 13, 14, 17, 18, 19, 21,  0,
  

In [35]:
ts.unique_classif([[1.4075008068002779,0.25378325414531328]])

[(0, 1)]

In [53]:
chr_estim_flat = np.array(ts.flatten(chr_estim))
chr_estim_flat_new = np.array(ts.flatten(chr_estim_new))
np.sum(chr_estim_flat[:,-1]!=chr_estim_flat_new[:,-1])
pos_dif = np.where(chr_estim_flat[:,-1]!=chr_estim_flat_new[:,-1])[0]

In [59]:
reind_old = np.argsort(chr_estim_flat[:,0])
reind_new = np.argsort(chr_estim_flat_new[:,0])
pos_dif = np.where(chr_estim_flat[reind_old,-1]!=chr_estim_flat_new[reind_new,-1])[0]

In [60]:
pos_dif

array([], dtype=int64)

In [54]:
import matplotlib.pyplot as plt
flat_ = chr_estim_flat
for i in [0,1]:
    x,y,z = flat_[flat_[:,-1]==i,:3].T
    plt.plot(x,y,'o')
for pos in pos_dif:
    x,y,z = flat_[pos,:3].T
    plt.plot([x],[y],'ro')
plt.show()

In [56]:
chr_estim_flat_new[pos],chr_estim_flat[pos]

(array([  0.09105405,   0.75620262,  -0.18331434,  88.        ,
         18.        ,   1.        ,   1.        ]),
 array([  0.53733261,  -0.31869798,   0.588431  ,  88.        ,
          0.        ,   0.        ]))

In [51]:
import matplotlib.pyplot as plt
flat_ = chr_estim_flat_new
for i in [0,1]:
    x,y,z = flat_[flat_[:,-1]==i,:3].T
    plt.plot(x,y,'o')
plt.show()

In [8]:
split_chr

[[array([0.3496437678138829, -0.23852404398710134, 0.74997597970551455, 99, 0], dtype=object)],
 [array([-0.22296103835982559, 0.45835915908589808, -0.2090516553835064, 99,
         14], dtype=object)]]

In [12]:
pts_partitioned_[2]

[array([0.18553405305226198, -0.52716214232740188, 0.5549414906245359, 2, 0], dtype=object),
 array([-0.088050912958783131, 0.74077052807902488, -0.25216482427925263, 2,
        16], dtype=object)]